# TP2 - Julien Vanbelle

## Imports

In [ ]:
import os
import yake
from collections import Counter
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image
from collections import defaultdict
import sys
import spacy
from spacy.lang.fr.examples import sentences
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
from tabulate import tabulate

year = 1890

In [ ]:
# Création d'une liste de mots à ignorer
ignored = set(["conseil communal", "conseil général", "conseil d'administration","conseil supérieur","conseil provincial"])
ignored

In [ ]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

In [ ]:
# Lister les Fichiers
data_path = "/Users/julienvanbelle/Documents/GitHub/tac/data/txt"
files = os.listdir(data_path)

In [ ]:
# Imprimer le nombre de fichiers identifiés
len(files)

In [ ]:
# Enlever les fichiers qui ne commencent pas par Bxl_
date_files = [f for f in files if f.startswith('Bxl_1890')]
len(date_files)

In [ ]:
for f in sorted(date_files):
    text = open(os.path.join(data_path, f), 'r').read()
    keywords = kw_extractor.extract_keywords(text.lower())  ##.lower() ==> permet de mettre le texte en miniscule
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2 and kw.lower() not in ignored:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

In [ ]:
# Stopwords (Idem que dans s1)
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu", "ville"]
sw = set(sw)

In [ ]:
# Stocker le contenu de ces fichiers dans une liste
content_list = []
for txt in date_files:
    with open(os.path.join(data_path, txt), 'r') as f:
        content_list.append(f.read())

len(content_list)

In [ ]:
# Ecrire tout le contenu dans un fichier temporaire
temp_path = '/Users/julienvanbelle/Documents/GitHub/tac/data'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'{year}.txt'), 'w') as f:
    f.write(' '.join(content_list))

In [ ]:
# Imprimer le contenu du fichier et constater les "déchets"
with open(os.path.join(temp_path, f'{year}.txt'), 'r') as f:
    before = f.read()

before[:500]

In [ ]:
def clean_text(year, folder=None):
    if folder is None:
        input_path = f"{year}.txt"
        output_path = f"{year}_clean.txt"
    else:
        input_path = f"{folder}/{year}.txt"
        output_path = f"{folder}/{year}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [ ]:
clean_text(year, folder=temp_path)

In [ ]:
# Vérifier le résultat
with open(os.path.join(temp_path, f'{year}_clean.txt'), 'r') as f:
    after = f.read()

after[:500]

In [ ]:
frequencies = Counter(after.split())
print(frequencies.most_common(10))

In [ ]:
cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"{year}.png"))
Image(filename=os.path.join(temp_path, f"{year}.png"))

In [ ]:
nlp = spacy.load('fr_core_news_md')

In [ ]:
# Charger le texte
n=900000
text = open("/Users/julienvanbelle/Documents/GitHub/tac/data/all.txt", encoding='utf-8').read()[:n]

In [ ]:
%%time
# Traiter le texte

doc = nlp(text)

In [ ]:
# Compter les entités
people = defaultdict(int)
organisation = defaultdict(int)
places = defaultdict(int)

for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1
    if ent.label_ == "ORG" and len(ent.text) > 3:
        organisation[ent.text] += 1
    if ent.label_ == "LOC" and len(ent.text) > 3:
        places[ent.text] += 1
      
    
      

In [ ]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)
sorted_organisation = sorted(organisation.items(), key=lambda kv: kv[1], reverse=True)
sorted_places = sorted(places.items(), key=lambda kv: kv[1], reverse=True)


for person, freq in sorted_people:
    print(f"{person} (PER)apparait {freq} fois dans le corpus")

for organisation, freq in sorted_organisation:
    print(f"{organisation} (ORG)apparait {freq} fois dans le corpus")

for places, freq in sorted_places:
    print(f"{places} (LOC)apparait {freq} fois dans le corpus")

In [ ]:
phrase_list = []

phrase1 = "C'est dire que vous avez des facultés et des aptitudes spéciales pour l'art auquel vous avez consacré votre vie."
phrase2 = "Les artistes musiciens et choristes du théâtre de l'Alhambra sollicitent un subside à l'effet de pouvoir organiser des concerts classiques et hebdomadaires."
phrase3 = "M. Yseux. J'ai posé la question de savoir si le fait est vrai ou faux."
phrase4 = "Dans ces conditions, nous espérons que la Députation permanente refusera de mandater d'office une dépense qui légalement nenous incombe pas. Au besoin, la justice sera appelée à statuer."
phrase5 = "Le directeur fait, à la fin de chaque mois, le dépouillement des certificats délivrés par les chefs de service aux internes et aux externes et le transmet au Secrétariat général."
phrase6 = "Il prépare les médicaments nécessaires et surveille l'administration de ceux-ci ; il ne peut modifier en rien les traitements indiqués par le médecin directeur, ni en instituer aucun autre, lorsque celui-ci est à l'hospice."
phrase7 = "Dans la dernière séance, nous avons voté poulies fêtes un crédit supplémentaire de 53,000 francs."
phrase8 = "Il était néanmoins difficile de la combler parce que, d'une part, les crédits votés par le Conseil communal devaient subvenir à de multiples emplois, et que, d'autre part, le crédit voté par les Chambres était des plus réduits."
phrase9 = "Nous ne pouvons mieux faire c o n n a î t r e la situation p r o s p è r e de l'Académie qu'en mentionnant le nombre d'élèves q u i ont é t é inscrits pour les cours d u soir, soit 7 5 9 , ce q u i , avec les 259 jeunes gens inscrits pour les cours du j o u r , forme un total de 1,018 élèves ; beaucoup d'entre eux se sont fait inscrire à la fois pour les cours du jour et du soir."
phrase10 = "Grâce à la généreuse intervention du fonds Bischoffsheim, l'ouverture de l'école a eu lieu le 9 septembre 1889."
phrase_list.append(phrase1)
phrase_list.append(phrase2)
phrase_list.append(phrase3)
phrase_list.append(phrase4)
phrase_list.append(phrase5)
phrase_list.append(phrase6)
phrase_list.append(phrase7)
phrase_list.append(phrase8)
phrase_list.append(phrase9)
phrase_list.append(phrase10)

##print(phrase_list[:10])
print("nb d'éléments dans la liste =",len(phrase_list))

In [ ]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
score_list = [['Polarity', 'Subjectivity']]
def get_sentiment(input_text):
    blob = tb(input_text)
    polarity, subjectivity = blob.sentiment
    polarity_perc = f"{100*abs(polarity):.0f}"
    subjectivity_perc = f"{100*subjectivity:.0f}"
    if polarity > 0:
        polarity_str = f"{polarity_perc}% positive"
    elif polarity < 0:
        polarity_str = f"{polarity_perc}% negative"
    else:
        polarity_str = "neutral"
    if subjectivity > 0:
        subjectivity_str = f"{subjectivity}% subjective"
    else:
        subjectivity_str = "perfectly objective"
    print(f"This text is {polarity_str} and {subjectivity_str}.")
    score_list.append([polarity_str,subjectivity_str])


In [ ]:


for y in range(0,len(phrase_list)):
 print("Phrase", y+1, "==" ,phrase_list[y])
 get_sentiment(phrase_list[y])
 print()
 print(score_list[y+1])
 print()


 

In [ ]:
print(tabulate(score_list,headers='firstrow',tablefmt='fancy_grid'))